# Innovation Modeling Kill Test
**Goal**: Train vanilla GRU h=64 with auxiliary innovation head for t1.

The innovation head predicts `t1_t - phi * t1_{t-1}` where phi is learnable (init 0.976).
Primary head outputs raw (t0, t1) as usual. Innovation is train-only regularizer.

**Kill test**: 3 seeds (s42, s43, s44). Compare val t1 vs base parity_v1 mean (0.2689).
- **Kill**: mean val avg < 0.2680
- **Pass**: mean val t1 improves > 0.003 over base

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup -- clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train innovation kill test (3 seeds)
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_parity_v1_innovation.yaml"
SEEDS = [42, 43, 44]

print("=== INNOVATION MODELING KILL TEST ===")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS}")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training innovation seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with rc={proc.returncode}")

print(f"\nAll training done!")

In [ ]:
# Cell 3: Evaluate innovation vs base
import os, glob, torch
os.chdir("/content/competition_package")

BASE_MEAN_VAL = 0.2689  # mean of 23 base parity_v1 seeds

# Collect innovation results
innovation_scores = []
for pt in sorted(glob.glob("logs/gru_parity_v1_innovation_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "N/A")
    
    # Extract phi value
    phi_val = float(ckpt["model_state_dict"].get("phi", 0.976))
    
    seed = basename.split("seed")[1].replace(".pt", "")
    innovation_scores.append((seed, score, epoch, phi_val))
    print(f"seed {seed}: val={score:.4f}, epoch={epoch}, phi={phi_val:.4f}")

if innovation_scores:
    scores = [s[1] for s in innovation_scores]
    mean_score = sum(scores) / len(scores)
    delta = mean_score - BASE_MEAN_VAL
    
    print(f"\n=== KILL TEST RESULT ===")
    print(f"Innovation mean val: {mean_score:.4f}")
    print(f"Base mean val:       {BASE_MEAN_VAL:.4f}")
    print(f"Delta:               {delta:+.4f}")
    print(f"Best:                {max(scores):.4f}")
    print(f"Worst:               {min(scores):.4f}")
    
    phis = [s[3] for s in innovation_scores]
    print(f"Phi values:          {[f'{p:.4f}' for p in phis]}")
    
    if mean_score < 0.2680:
        print(f"\n>>> KILL: mean {mean_score:.4f} < 0.2680 threshold")
    elif delta > 0.003:
        print(f"\n>>> PASS: delta {delta:+.4f} > 0.003 threshold -> SCALE UP!")
    else:
        print(f"\n>>> MARGINAL: delta {delta:+.4f}. Decide manually.")
else:
    print("No innovation checkpoints found!")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/gru_parity_v1_innovation_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

shutil.make_archive("/content/innovation_kill_test", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/innovation_kill_test.zip") / 1e6
print(f"\ninnovation_kill_test.zip: {sz:.1f}MB")

shutil.copy("/content/innovation_kill_test.zip",
            "/content/drive/MyDrive/wunderfund/innovation_kill_test.zip")
print("Saved to Drive: MyDrive/wunderfund/innovation_kill_test.zip")